In [ ]:
# (1) Install JDK
!sudo apt update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://cli.github.com/packages stable/main amd64 Packages [355 B]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,759 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [4,070 kB]
Get:14 htt

In [ ]:
# (2) Install Spark — robust download
!rm -f spark-3.5.6-bin-hadoop3.tgz

!curl -L --retry 5 --retry-delay 5 \
  -o spark-3.5.6-bin-hadoop3.tgz \
  https://archive.apache.org/dist/spark/spark-3.5.6/spark-3.5.6-bin-hadoop3.tgz

!tar -xzf spark-3.5.6-bin-hadoop3.tgz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  382M  100  382M    0     0  10.9M      0  0:00:34  0:00:34 --:--:-- 7542k


In [ ]:
# (3) Install pyspark
!pip install -q pyspark==4.0.0 py4j findspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 15.0 MB/s eta 0:00:00


In [ ]:
# (4) Setup Environment variables
import findspark
findspark.init("/content/spark-3.5.6-bin-hadoop3")



#Create Spark Session

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("week 6 inclass activity").getOrCreate()
spark


#CreateDF

In [ ]:
#using lists
df = spark.createDataFrame(
    [(24, "Tom"), (23, "Alice"), (26, "Bob")],
     ["age", "name"]
)

In [ ]:
df.show()

+---+-----+
|age| name|
+---+-----+
| 24|  Tom|
| 23|Alice|
| 26|  Bob|
+---+-----+



In [ ]:
#using dictionary (key-value)

df1 = spark.createDataFrame(
    [
        {'name': 'Tom', 'age': 24},
        {'name': 'Alice', 'age': 23},
        {'name': 'Bob', 'age': 26}
     ])
df1.show()

+---+-----+
|age| name|
+---+-----+
| 24|  Tom|
| 23|Alice|
| 26|  Bob|
+---+-----+



In [ ]:
data = [("Alice", 25), ("Bob", 30), ("Charlie", 35)]
columns = ["name", "age"]

# Create a DataFrame
df2 = spark.createDataFrame(data, columns)
df2.show()

+-------+---+
|   name|age|
+-------+---+
|  Alice| 25|
|    Bob| 30|
|Charlie| 35|
+-------+---+



In [ ]:
df2.count()

3

#createTempView

In [ ]:
df.createOrReplaceTempView('people')

In [ ]:
spark.catalog.listTables()

[Table(name='people', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [ ]:
res = spark.sql('select  * from people')
res.show()

+---+-----+
|age| name|
+---+-----+
| 24|  Tom|
| 23|Alice|
| 26|  Bob|
+---+-----+



#Distinct

In [ ]:
df = spark.createDataFrame(
    [(24, "Tom"), (23, "Alice"), (23, "Alice")], ["age", "name"])
df.show()

+---+-----+
|age| name|
+---+-----+
| 24|  Tom|
| 23|Alice|
| 23|Alice|
+---+-----+



In [ ]:
#drops duplicate rows
df.distinct().show()

+---+-----+
|age| name|
+---+-----+
| 24|  Tom|
| 23|Alice|
+---+-----+



In [ ]:
#res.show()

+---+---+
| C1| C2|
+---+---+
|  a|  1|
|  a|  1|
|  b|  3|
+---+---+



#Select

In [ ]:
df.select('name').distinct().show()

+-----+
| name|
+-----+
|  Tom|
|Alice|
+-----+



In [ ]:
df = spark.createDataFrame(
    [(24, "Tom", "M"), (23, "Alice", "F"), (23, "Alice", "F"), (25, "Tom", None)],
    ["age", "name", "gender"])
df.show()

+---+-----+------+
|age| name|gender|
+---+-----+------+
| 24|  Tom|     M|
| 23|Alice|     F|
| 23|Alice|     F|
| 25|  Tom|  NULL|
+---+-----+------+



In [ ]:
df.isEmpty()

False

In [ ]:
df.select('*').show()

+---+-----+------+
|age| name|gender|
+---+-----+------+
| 24|  Tom|     M|
| 23|Alice|     F|
| 23|Alice|     F|
| 25|  Tom|  NULL|
+---+-----+------+



In [ ]:
df.select('age', 'gender').show()

+---+------+
|age|gender|
+---+------+
| 24|     M|
| 23|     F|
| 23|     F|
| 25|  NULL|
+---+------+



In [ ]:
res = df.select(['name', 'gender'])
res.show()

+-----+------+
| name|gender|
+-----+------+
|  Tom|     M|
|Alice|     F|
|Alice|     F|
|  Tom|  NULL|
+-----+------+



#Drop

In [ ]:
res = df.drop('gender')
res.show()

+---+-----+
|age| name|
+---+-----+
| 24|  Tom|
| 23|Alice|
| 23|Alice|
| 25|  Tom|
+---+-----+



In [ ]:
df.show()

+---+-----+------+
|age| name|gender|
+---+-----+------+
| 24|  Tom|     M|
| 23|Alice|     F|
| 23|Alice|     F|
| 25|  Tom|  NULL|
+---+-----+------+



#dropDuplicates

In [ ]:
from pyspark.sql import Row

In [ ]:
from pyspark.sql import Row
df = spark.createDataFrame([
    Row(name='Alice', age=25, height=80),     # {'name': 'Tom', 'age': 24}
    Row(name='Alice', age=25, height=80),
    Row(name='Alice', age=30, height=80)
])
df.show()

+-----+---+------+
| name|age|height|
+-----+---+------+
|Alice| 25|    80|
|Alice| 25|    80|
|Alice| 30|    80|
+-----+---+------+



In [ ]:
df.distinct().show()

+-----+---+------+
| name|age|height|
+-----+---+------+
|Alice| 25|    80|
|Alice| 30|    80|
+-----+---+------+



In [ ]:
df.dropDuplicates().show()

+-----+---+------+
| name|age|height|
+-----+---+------+
|Alice| 25|    80|
|Alice| 30|    80|
+-----+---+------+



In [ ]:
df.dropDuplicates(['name']).show()

+-----+---+------+
| name|age|height|
+-----+---+------+
|Alice| 25|    80|
+-----+---+------+



#fillna

In [ ]:
df = spark.createDataFrame(
      [
        (30, 80.5, "Alice", None),
        (25, None, "Bob", None),
        (None, None, "Tom", None),
        (None, None, None, True)
      ],
    schema=["age", "height", "name", "flag"])
df.show()

+----+------+-----+----+
| age|height| name|flag|
+----+------+-----+----+
|  30|  80.5|Alice|NULL|
|  25|  NULL|  Bob|NULL|
|NULL|  NULL|  Tom|NULL|
|NULL|  NULL| NULL|true|
+----+------+-----+----+



In [ ]:
df.fillna(-1).show()

+---+------+-----+----+
|age|height| name|flag|
+---+------+-----+----+
| 30|  80.5|Alice|NULL|
| 25|  -1.0|  Bob|NULL|
| -1|  -1.0|  Tom|NULL|
| -1|  -1.0| NULL|true|
+---+------+-----+----+



In [ ]:
df.fillna(False).show()

+----+------+-----+-----+
| age|height| name| flag|
+----+------+-----+-----+
|  30|  80.5|Alice|false|
|  25|  NULL|  Bob|false|
|NULL|  NULL|  Tom|false|
|NULL|  NULL| NULL| true|
+----+------+-----+-----+



In [ ]:
df.fillna({'age': -1, 'height': 0, 'name': 'unknown', 'flag': False}).show()

+---+------+-------+-----+
|age|height|   name| flag|
+---+------+-------+-----+
| 30|  80.5|  Alice|false|
| 25|   0.0|    Bob|false|
| -1|   0.0|    Tom|false|
| -1|   0.0|unknown| true|
+---+------+-------+-----+



#Filter

In [ ]:
df = spark.createDataFrame([
    (22, "Alice", "Math"), (25, "Bob", "Physics"), (27, "Charlie", "Chemistry")],
    schema=["age", "name", "subject"])

df.show()

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 22|  Alice|     Math|
| 25|    Bob|  Physics|
| 27|Charlie|Chemistry|
+---+-------+---------+



###Filter by Column instances.



In [ ]:
df.filter(df.age > 23).show()

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 25|    Bob|  Physics|
| 27|Charlie|Chemistry|
+---+-------+---------+



###Filter by SQL expression in a string

In [ ]:
df.filter('age > 23').show()

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 25|    Bob|  Physics|
| 27|Charlie|Chemistry|
+---+-------+---------+



In [ ]:
df.where('age = 25').show()

+---+----+-------+
|age|name|subject|
+---+----+-------+
| 25| Bob|Physics|
+---+----+-------+



###Filter by multiple conditions

In [ ]:
df.filter((df.age > 23) & (df.subject == 'Physics') ).show()

+---+----+-------+
|age|name|subject|
+---+----+-------+
| 25| Bob|Physics|
+---+----+-------+



In [ ]:
df.filter((df.age == 25) | (df.subject == 'Chemistry') ).show()

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 25|    Bob|  Physics|
| 27|Charlie|Chemistry|
+---+-------+---------+



###Filter by multiple conditions using SQL expression

In [ ]:
df.filter('age > 23 AND name = "Bob" ').show()

+---+----+-------+
|age|name|subject|
+---+----+-------+
| 25| Bob|Physics|
+---+----+-------+



In [ ]:
df.filter('age > 23 OR name = "Bob" ').show()

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 25|    Bob|  Physics|
| 27|Charlie|Chemistry|
+---+-------+---------+



###Filter using the Column.isin() function.

In [ ]:
df.filter(df.subject.isin("Math","Physics")).show()

+---+-----+-------+
|age| name|subject|
+---+-----+-------+
| 22|Alice|   Math|
| 25|  Bob|Physics|
+---+-----+-------+



###Filter by a list of values using the Column.isin() function.

In [ ]:
df.filter(df.subject.isin(["Math","Physics"])).show()

+---+-----+-------+
|age| name|subject|
+---+-----+-------+
| 22|Alice|   Math|
| 25|  Bob|Physics|
+---+-----+-------+



###Fliter to exclude

In [ ]:
df.filter(~df.subject.isin(["Math","Physics"])).show()

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 27|Charlie|Chemistry|
+---+-------+---------+



###Filter using the Column.isNotNull() function

In [ ]:
df.filter(df.name.isNotNull()).show()

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 22|  Alice|     Math|
| 25|    Bob|  Physics|
| 27|Charlie|Chemistry|
+---+-------+---------+



###Filter using the Column.like() function

In [ ]:
df.filter(df.name.like('A%')).show()

+---+-----+-------+
|age| name|subject|
+---+-----+-------+
| 22|Alice|   Math|
+---+-----+-------+



In [ ]:
df.filter(df.name.like('%i%')).show()  # % : used to capture any substring (zero or more characters)

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 22|  Alice|     Math|
| 27|Charlie|Chemistry|
+---+-------+---------+



In [ ]:
df.filter(df.name.like('%i_')).show()  # _ : captures exactly any one character

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 27|Charlie|Chemistry|
+---+-------+---------+



###Filter using the Column.contains() function

In [ ]:
df.filter(df.name.contains('i')).show()

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 22|  Alice|     Math|
| 27|Charlie|Chemistry|
+---+-------+---------+



###Filter using the Column.between() function.

In [ ]:
df.filter(df.age.between(25, 55)).show()

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 25|    Bob|  Physics|
| 27|Charlie|Chemistry|
+---+-------+---------+



#First

In [ ]:
df.first()

Row(age=22, name='Alice', subject='Math')

#head

In [ ]:
df.head(2)

[Row(age=22, name='Alice', subject='Math'),
 Row(age=25, name='Bob', subject='Physics')]

#Tail

In [ ]:
df.tail(2)

[Row(age=25, name='Bob', subject='Physics'),
 Row(age=27, name='Charlie', subject='Chemistry')]

#Limit

In [ ]:
df.limit(2).show()

+---+-----+-------+
|age| name|subject|
+---+-----+-------+
| 22|Alice|   Math|
| 25|  Bob|Physics|
+---+-----+-------+



#offset

In [ ]:
df.offset(1).limit(1).show()

+---+----+-------+
|age|name|subject|
+---+----+-------+
| 25| Bob|Physics|
+---+----+-------+



#orderby

In [ ]:
df.show()

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 22|  Alice|     Math|
| 25|    Bob|  Physics|
| 27|Charlie|Chemistry|
+---+-------+---------+



In [ ]:
df.orderBy(df.age.desc()).show()

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 27|Charlie|Chemistry|
| 25|    Bob|  Physics|
| 22|  Alice|     Math|
+---+-------+---------+



In [ ]:
df.orderBy(df.subject, df.name).show()

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 27|Charlie|Chemistry|
| 22|  Alice|     Math|
| 25|    Bob|  Physics|
+---+-------+---------+



In [ ]:
df.sort(df.age.desc()).show()

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 27|Charlie|Chemistry|
| 25|    Bob|  Physics|
| 22|  Alice|     Math|
+---+-------+---------+



In [ ]:
df.sort('age', ascending=False).show()

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 27|Charlie|Chemistry|
| 25|    Bob|  Physics|
| 22|  Alice|     Math|
+---+-------+---------+



#Agg

In [ ]:
df = spark.createDataFrame([
    (22, "Alice", "Math"), (25, "Bob", "Physics"), (27, "Charlie", "Chemistry")],
    schema=["age", "name", "subject"])

df.show()

+---+-------+---------+
|age|   name|  subject|
+---+-------+---------+
| 22|  Alice|     Math|
| 25|    Bob|  Physics|
| 27|Charlie|Chemistry|
+---+-------+---------+



In [ ]:
df.agg({'age': 'max'}).show()

+--------+
|max(age)|
+--------+
|      27|
+--------+



In [ ]:
df.agg({'age': 'min'}).show()

+--------+
|min(age)|
+--------+
|      22|
+--------+



In [ ]:
#multiple
from pyspark.sql import functions as f

In [ ]:
df.agg(
      f.min('age').alias('min_age'),
      f.max('age').alias('max_age'),
      f.avg('age').alias('avg_age'),
      f.sum('age').alias('total_age'),
      f.count('age').alias('count_age'),
).show()

+-------+-------+------------------+---------+---------+
|min_age|max_age|           avg_age|total_age|count_age|
+-------+-------+------------------+---------+---------+
|     22|     27|24.666666666666668|       74|        3|
+-------+-------+------------------+---------+---------+



#GroupBy

In [ ]:
df = spark.createDataFrame([
    ("Alice", 22), ("Bob", 22), ("Alice", 32), ("Bob", 22), ("Bob", 25)], schema=["name", "age"])

df.show()

+-----+---+
| name|age|
+-----+---+
|Alice| 22|
|  Bob| 22|
|Alice| 32|
|  Bob| 22|
|  Bob| 25|
+-----+---+



### Empty grouping columns triggers a global aggregation

In [ ]:
df.agg({'age': 'avg'}).show()

+--------+
|avg(age)|
+--------+
|    24.6|
+--------+



###Group-by ‘name’, and specify a dictionary to calculate the summation of ‘age’

In [ ]:
df.groupBy('name').agg({'age': 'avg'}).show()

+-----+--------+
| name|avg(age)|
+-----+--------+
|  Bob|    23.0|
|Alice|    27.0|
+-----+--------+



#Group-by ‘name’, and calculate maximum values.

In [ ]:
df.groupBy(df.name).max().sort("name").show()  #max on all columns except the groupby column

+-----+--------+
| name|max(age)|
+-----+--------+
|Alice|      32|
|  Bob|      25|
+-----+--------+



In [ ]:
df.groupBy(df.name).agg({"age": "max"}).show() #explicitly specify on which column to apply the max function

+-----+--------+
| name|max(age)|
+-----+--------+
|  Bob|      25|
|Alice|      32|
+-----+--------+



###Group-by ‘name’ and ‘age’, and calculate the number of rows in each group.



In [ ]:
df.show()

+-----+---+
| name|age|
+-----+---+
|Alice| 22|
|  Bob| 22|
|Alice| 32|
|  Bob| 22|
|  Bob| 25|
+-----+---+



In [ ]:
df.groupBy(['name', 'age']).count().show()

+-----+---+-----+
| name|age|count|
+-----+---+-----+
|  Bob| 22|    2|
|Alice| 22|    1|
|Alice| 32|    1|
|  Bob| 25|    1|
+-----+---+-----+



#intersect

In [ ]:
df1 = spark.createDataFrame([("a", 1), ("a", 1), ("b", 3), ("c", 4)], ["C1", "C2"])
df2 = spark.createDataFrame([("b", 3), ("a", 1), ("a", 1)], ["C21", "C22"])

In [ ]:
res = df1.intersect(df2)
res.show()

+---+---+
| C1| C2|
+---+---+
|  b|  3|
|  a|  1|
+---+---+



In [ ]:
res = df1.intersectAll(df2)
res.show()

+---+---+
| C1| C2|
+---+---+
|  a|  1|
|  a|  1|
|  b|  3|
+---+---+



#Union

In [ ]:
df1 = spark.createDataFrame([(1, 'A'), (2, 'B')], ['id', 'value'])
df2 = spark.createDataFrame([(3, 'C'), (4, 'D')], ['id', 'value'])


In [ ]:
df3 = df1.union(df2)
df3.show()


+---+-----+
| id|value|
+---+-----+
|  1|    A|
|  2|    B|
|  3|    C|
|  4|    D|
+---+-----+



###add a new row

In [ ]:
df1.show()

+---+-----+
| id|value|
+---+-----+
|  1|    A|
|  2|    B|
+---+-----+



In [ ]:
newRow = spark.createDataFrame([(10,'Z')], ['id', 'value'])

df4 = df1.union(newRow)
df4.show()

+---+-----+
| id|value|
+---+-----+
|  1|    A|
|  2|    B|
| 10|    Z|
+---+-----+



#Subtract

In [ ]:
df1 = spark.createDataFrame([(1, 'A'), (2, 'B'), (3, 'C')], ['id', 'value'])
df2 = spark.createDataFrame([(3, 'C'), (4, 'D')], ['id', 'value'])

In [ ]:
df1.show()

+---+-----+
| id|value|
+---+-----+
|  1|    A|
|  2|    B|
|  3|    C|
+---+-----+



In [ ]:
df2.show()

+---+-----+
| id|value|
+---+-----+
|  3|    C|
|  4|    D|
+---+-----+



In [ ]:
df1.subtract(df2).show()

+---+-----+
| id|value|
+---+-----+
|  2|    B|
|  1|    A|
+---+-----+



In [ ]:
df2.subtract(df1).show()

+---+-----+
| id|value|
+---+-----+
|  4|    D|
+---+-----+



#Join

In [ ]:
df1 = spark.createDataFrame([Row(name="Alice", age=22), Row(name="Bob", age=25)])
df2 = spark.createDataFrame([Row(name="Tom", height=80), Row(name="Bob", height=85)])

df3 = spark.createDataFrame([
    Row(name="Alice", age=30, height=80),
    Row(name="Bob", age=25, height=None),
    Row(name="Tom", age=None, height=None),
    Row(name=None, age=None, height=None),
])

In [ ]:
df1.show()

+-----+---+
| name|age|
+-----+---+
|Alice| 22|
|  Bob| 25|
+-----+---+



In [ ]:
df2.show()

+----+------+
|name|height|
+----+------+
| Tom|    80|
| Bob|    85|
+----+------+



In [ ]:
df1.join(df2, on='name', how='inner').show()

+----+---+------+
|name|age|height|
+----+---+------+
| Bob| 25|    85|
+----+---+------+



In [ ]:
df1.join(df2, on='name', how='leftouter').show()

+-----+---+------+
| name|age|height|
+-----+---+------+
|Alice| 22|  NULL|
|  Bob| 25|    85|
+-----+---+------+



In [ ]:
df1.join(df2, on='name', how='rightouter').show()

+----+----+------+
|name| age|height|
+----+----+------+
| Tom|NULL|    80|
| Bob|  25|    85|
+----+----+------+



In [ ]:
df3.show()

+-----+----+------+
| name| age|height|
+-----+----+------+
|Alice|  30|    80|
|  Bob|  25|  NULL|
|  Tom|NULL|  NULL|
| NULL|NULL|  NULL|
+-----+----+------+



In [ ]:
df1.join(df3, on='name', how='inner').show()

+-----+---+---+------+
| name|age|age|height|
+-----+---+---+------+
|Alice| 22| 30|    80|
|  Bob| 25| 25|  NULL|
+-----+---+---+------+



#Replace

In [ ]:
df = spark.createDataFrame([
    (10, 80, "Alice"),
    (5, None, "Bob"),
    (None, 10, "Tom"),
    (None, None, None)],
    schema=["age", "height", "name"])

df.show()

+----+------+-----+
| age|height| name|
+----+------+-----+
|  10|    80|Alice|
|   5|  NULL|  Bob|
|NULL|    10|  Tom|
|NULL|  NULL| NULL|
+----+------+-----+



###Replace 10 to 18 in the age column.

In [ ]:
df.replace(10, 20).show()

+----+------+-----+
| age|height| name|
+----+------+-----+
|  20|    80|Alice|
|   5|  NULL|  Bob|
|NULL|    20|  Tom|
|NULL|  NULL| NULL|
+----+------+-----+



In [ ]:
df.replace(10, 20, 'age').show()

+----+------+-----+
| age|height| name|
+----+------+-----+
|  20|    80|Alice|
|   5|  NULL|  Bob|
|NULL|    10|  Tom|
|NULL|  NULL| NULL|
+----+------+-----+



### Replace ‘Alice’ to ‘A’, and ‘Bob’ to ‘B’ in the ‘name’ column

In [ ]:
df.na.replace(['Alice', 'Bob'], ['A', 'B'], 'name').show()

+----+------+----+
| age|height|name|
+----+------+----+
|  10|    80|   A|
|   5|  NULL|   B|
|NULL|    10| Tom|
|NULL|  NULL|NULL|
+----+------+----+



#Statistics

In [ ]:
df = spark.createDataFrame(
    [("Bob", 23, 40.3, 150.5), ("Alice", 22, 37.8, 142.3), ("Tom", 21, 44.1, 142.2)],
    ["name", "age", "weight", "height"],
)
df.show()

+-----+---+------+------+
| name|age|weight|height|
+-----+---+------+------+
|  Bob| 23|  40.3| 150.5|
|Alice| 22|  37.8| 142.3|
|  Tom| 21|  44.1| 142.2|
+-----+---+------+------+



###Describe

In [ ]:
df.describe().show()

+-------+-----+----+------------------+-----------------+
|summary| name| age|            weight|           height|
+-------+-----+----+------------------+-----------------+
|  count|    3|   3|                 3|                3|
|   mean| NULL|22.0|40.733333333333334|            145.0|
| stddev| NULL| 1.0| 3.172275734127371|4.763402145525822|
|    min|Alice|  21|              37.8|            142.2|
|    max|  Tom|  23|              44.1|            150.5|
+-------+-----+----+------------------+-----------------+



###Summary

In [ ]:
df.summary().show()

+-------+-----+----+------------------+-----------------+
|summary| name| age|            weight|           height|
+-------+-----+----+------------------+-----------------+
|  count|    3|   3|                 3|                3|
|   mean| NULL|22.0|40.733333333333334|            145.0|
| stddev| NULL| 1.0| 3.172275734127371|4.763402145525822|
|    min|Alice|  21|              37.8|            142.2|
|    25%| NULL|  21|              37.8|            142.2|
|    50%| NULL|  22|              40.3|            142.3|
|    75%| NULL|  23|              44.1|            150.5|
|    max|  Tom|  23|              44.1|            150.5|
+-------+-----+----+------------------+-----------------+



In [ ]:
df.select('weight', 'height').summary('count', 'min', '75%').show()

+-------+------+------+
|summary|weight|height|
+-------+------+------+
|  count|     3|     3|
|    min|  37.8| 142.2|
|    75%|  44.1| 150.5|
+-------+------+------+



#Schema

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType

In [ ]:
df = spark.createDataFrame([("a", 1)], ["col1", "col2"])
df.show()

+----+----+
|col1|col2|
+----+----+
|   a|   1|
+----+----+



In [ ]:
df.schema

StructType([StructField('col1', StringType(), True), StructField('col2', LongType(), True)])

In [ ]:
myschema = StructType(
      [
          StructField('col1', StringType()),
          StructField('col2', StringType())
      ]
)

#to

In [ ]:
df2 = df.to(myschema)
df2.schema

StructType([StructField('col1', StringType(), True), StructField('col2', StringType(), True)])

#toDF

In [ ]:
df = spark.createDataFrame([(24, "Tom"), (23, "Alice"),
    (26, "Bob")], ["age", "name"])

df.show()

+---+-----+
|age| name|
+---+-----+
| 24|  Tom|
| 23|Alice|
| 26|  Bob|
+---+-----+



In [ ]:
df2 = df.toDF('C1', 'C2')

In [ ]:
df2.show()

+---+-----+
| C1|   C2|
+---+-----+
| 24|  Tom|
| 23|Alice|
| 26|  Bob|
+---+-----+



#withColumn

In [ ]:
df = spark.createDataFrame([(2, "Alice"), (5, "Bob")], schema=["id", "name"])
df.show()

+---+-----+
| id| name|
+---+-----+
|  2|Alice|
|  5|  Bob|
+---+-----+



In [ ]:
#adds a new column
df.withColumn('newid', df.id + 500).show()

+---+-----+-----+
| id| name|newid|
+---+-----+-----+
|  2|Alice|  502|
|  5|  Bob|  505|
+---+-----+-----+



#withColumns

In [ ]:
df = spark.createDataFrame([(2, "Alice"), (5, "Bob")], schema=["id", "name"])
df.show()

+---+-----+
| id| name|
+---+-----+
|  2|Alice|
|  5|  Bob|
+---+-----+



In [ ]:
#add numtiple columns

df.withColumns({'newid': df.id + 100, 'fullname':  f.concat(df.id, df.name)}).show()

+---+-----+-----+--------+
| id| name|newid|fullname|
+---+-----+-----+--------+
|  2|Alice|  102|  2Alice|
|  5|  Bob|  105|    5Bob|
+---+-----+-----+--------+



#withColumnRenamed

In [ ]:
df = spark.createDataFrame([(2, "Alice"), (5, "Bob")], schema=["id", "name"])
df.show()

+---+-----+
| id| name|
+---+-----+
|  2|Alice|
|  5|  Bob|
+---+-----+



In [ ]:
df.withColumnRenamed('id', 'newid').show()

+-----+-----+
|newid| name|
+-----+-----+
|    2|Alice|
|    5|  Bob|
+-----+-----+



In [ ]:
df.withColumnRenamed('age', 'newage').show()

+---+-----+
| id| name|
+---+-----+
|  2|Alice|
|  5|  Bob|
+---+-----+



#withColumnsRenamed

In [ ]:
df = spark.createDataFrame([(2, "Alice"), (5, "Bob")], schema=["id", "name"])
df.show()

+---+-----+
| id| name|
+---+-----+
|  2|Alice|
|  5|  Bob|
+---+-----+



In [ ]:
df.withColumnsRenamed({"id": "newid", "name": "fullname"}).show()


+-----+--------+
|newid|fullname|
+-----+--------+
|    2|   Alice|
|    5|     Bob|
+-----+--------+



#When , Otherwise

In [ ]:
data = [("James","Male",60000),("Michael","Male",70000),
        ("Robert",None,400000),("Maria","Female",500000),
        ("Kim","Female",None), ("Dave","Unknown",None)]

columns = ["name","gender","salary"]

In [ ]:
df = spark.createDataFrame(data = data, schema = columns)
df.show()

+-------+-------+------+
|   name| gender|salary|
+-------+-------+------+
|  James|   Male| 60000|
|Michael|   Male| 70000|
| Robert|   NULL|400000|
|  Maria| Female|500000|
|    Kim| Female|  NULL|
|   Dave|Unknown|  NULL|
+-------+-------+------+



In [ ]:
from pyspark.sql.functions import when

In [ ]:
df2 = df.withColumn('gender_info',
              when(df.gender=='Male', 'M')
              .when(df.gender=='Female', 'F')
              .when(df.gender=='Unknown', 'U')
              .when(df.gender.isNull(), 'NA')
              .otherwise(df.gender)
              )

df2.show()

+-------+-------+------+-----------+
|   name| gender|salary|gender_info|
+-------+-------+------+-----------+
|  James|   Male| 60000|          M|
|Michael|   Male| 70000|          M|
| Robert|   NULL|400000|         NA|
|  Maria| Female|500000|          F|
|    Kim| Female|  NULL|          F|
|   Dave|Unknown|  NULL|          U|
+-------+-------+------+-----------+



#SQL - Expr()

In [ ]:
'''
CASE
    WHEN condition1 THEN result_value1
    WHEN condition2 THEN result_value2
    -----
    -----
    ELSE default-value
END;
'''


'\nCASE\n    WHEN condition1 THEN result_value1\n    WHEN condition2 THEN result_value2\n    -----\n    -----\n    ELSE default-value\nEND;\n'

In [ ]:
from pyspark.sql.functions import expr

In [ ]:
df3 = df2.withColumn('salary_bracket_info',

                  expr("CASE\
                        WHEN salary >=100000 THEN 'Rich Income'\
                        WHEN salary >=50000 THEN 'Medium Income'\
                        ELSE 'Average'\
                      END"
                  )
              )
df3.show()

+-------+-------+------+-----------+-------------------+
|   name| gender|salary|gender_info|salary_bracket_info|
+-------+-------+------+-----------+-------------------+
|  James|   Male| 60000|          M|      Medium Income|
|Michael|   Male| 70000|          M|      Medium Income|
| Robert|   NULL|400000|         NA|        Rich Income|
|  Maria| Female|500000|          F|        Rich Income|
|    Kim| Female|  NULL|          F|            Average|
|   Dave|Unknown|  NULL|          U|            Average|
+-------+-------+------+-----------+-------------------+



#Explode vs flatten

In [ ]:
data = [
    (1, [[10, 20], [30, 40]]),
    (2, [[50], [60, 70, 80]]),
    (3, [[7]])
]

In [ ]:
df = spark.createDataFrame(data, ["id", "nested_info"])
df.show()

+---+--------------------+
| id|         nested_info|
+---+--------------------+
|  1|[[10, 20], [30, 40]]|
|  2|[[50], [60, 70, 80]]|
|  3|               [[7]]|
+---+--------------------+



In [ ]:
from pyspark.sql import functions as f

In [ ]:
df2 = df.select('*', f.flatten(df.nested_info).alias('flattened_array'))
df2.show()

+---+--------------------+----------------+
| id|         nested_info| flattened_array|
+---+--------------------+----------------+
|  1|[[10, 20], [30, 40]]|[10, 20, 30, 40]|
|  2|[[50], [60, 70, 80]]|[50, 60, 70, 80]|
|  3|               [[7]]|             [7]|
+---+--------------------+----------------+



In [ ]:
df3= df.select('*', f.explode(df.nested_info).alias('exploded_array'))
df3.show()

+---+--------------------+--------------+
| id|         nested_info|exploded_array|
+---+--------------------+--------------+
|  1|[[10, 20], [30, 40]]|      [10, 20]|
|  1|[[10, 20], [30, 40]]|      [30, 40]|
|  2|[[50], [60, 70, 80]]|          [50]|
|  2|[[50], [60, 70, 80]]|  [60, 70, 80]|
|  3|               [[7]]|           [7]|
+---+--------------------+--------------+



In [ ]:
df4 = df3.select('*', f.explode(df3.exploded_array).alias('exploded_again'))
df4.show()

+---+--------------------+--------------+--------------+
| id|         nested_info|exploded_array|exploded_again|
+---+--------------------+--------------+--------------+
|  1|[[10, 20], [30, 40]]|      [10, 20]|            10|
|  1|[[10, 20], [30, 40]]|      [10, 20]|            20|
|  1|[[10, 20], [30, 40]]|      [30, 40]|            30|
|  1|[[10, 20], [30, 40]]|      [30, 40]|            40|
|  2|[[50], [60, 70, 80]]|          [50]|            50|
|  2|[[50], [60, 70, 80]]|  [60, 70, 80]|            60|
|  2|[[50], [60, 70, 80]]|  [60, 70, 80]|            70|
|  2|[[50], [60, 70, 80]]|  [60, 70, 80]|            80|
|  3|               [[7]]|           [7]|             7|
+---+--------------------+--------------+--------------+

